In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)      #input=1*28*28, Output=16*28*28
        self.BN1   = nn.BatchNorm2d(16)                  #input=16*28*28, Output=16*28*28
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)     #input=16*28*28, Output=32*28*28
        self.drop1 = nn.Dropout(0.25)                    #input=32*28*28, Output=32*28*28
        self.pool1 = nn.MaxPool2d(2, 2)                  #input=32*28*28, Output=32*14*14
        
        self.conv3 = nn.Conv2d(32, 16, 3, padding=1)     #input=32*14*14, Output=16*14*14
        self.BN2   = nn.BatchNorm2d(16)                  #input=16*14*14, Output=16*14*14
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)     #input=16*14*14, Output=32*14*14
        self.drop2 = nn.Dropout(0.25)                    #input=32*14*14, Output=32*14*14
        self.pool2 = nn.MaxPool2d(2, 2)                  #input=32*14*14, Output=32*7*7

        self.conv5 = nn.Conv2d(32, 12, 3, padding=1)     #input=32*7*7, Output=14*7*7
        self.BN3   = nn.BatchNorm2d(12)                  #input=14*7*7, Output=14*7*7
        self.drop3 = nn.Dropout(0.25)                    #input=14*7*7, Output=14*7*7
        self.pool3 = nn.MaxPool2d(2, 2)                  #input=14*7*7, Output=14*3*3

        self.conv6 = nn.Conv2d(12, 10, 3)                #input=14*3*3, Output=10*1*1
        self.gap1  = nn.AdaptiveAvgPool2d((1,1))         #input=10*1*1, Output=10*1*1
        self.fc1    = nn.Linear(10*1*1, out_features=10) #input=10*1*1, Output=10
             

    def forward(self, x):
        
        x = self.pool1(self.drop1(F.relu(self.conv2(F.relu(self.BN1(self.conv1(x)))))))
        x = self.pool2(self.drop2(F.relu(self.conv4(F.relu(self.BN2(self.conv3(x)))))))
        x = self.pool3(self.drop3(F.relu(self.BN3(self.conv5(x)))))
        x = F.relu(self.conv6(x))      

        x = self.gap1(x)
        x = x.view(-1, 10)
        x = self.fc1(x)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 32, 28, 28]           4,640
           Dropout-4           [-1, 32, 28, 28]               0
         MaxPool2d-5           [-1, 32, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           4,624
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 32, 14, 14]           4,640
           Dropout-9           [-1, 32, 14, 14]               0
        MaxPool2d-10             [-1, 32, 7, 7]               0
           Conv2d-11             [-1, 12, 7, 7]           3,468
      BatchNorm2d-12             [-1, 12, 7, 7]              24
          Dropout-13             [-1, 12, 7, 7]               0
        MaxPool2d-14             [-1, 1

<ipython-input-14-db30d2dbf7e0>:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [16]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [17]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: epoch: {}, Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 14):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-14-db30d2dbf7e0>:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.09743601828813553 batch_id=468: 100%|██████████| 469/469 [01:40<00:00,  4.68it/s]



Test set: epoch: 1, Average loss: 0.1345, Accuracy: 9685/10000 (96.8%)



loss=0.13191278278827667 batch_id=468: 100%|██████████| 469/469 [01:39<00:00,  4.73it/s]



Test set: epoch: 2, Average loss: 0.0692, Accuracy: 9876/10000 (98.8%)



loss=0.020905466750264168 batch_id=468: 100%|██████████| 469/469 [01:39<00:00,  4.69it/s]


...


Test set: epoch: 11, Average loss: 0.0332, Accuracy: 9928/10000 (99.3%)



loss=0.005301088094711304 batch_id=468: 100%|██████████| 469/469 [01:37<00:00,  4.79it/s]



Test set: epoch: 12, Average loss: 0.0289, Accuracy: 9940/10000 (99.4%)



loss=0.010571555234491825 batch_id=468: 100%|██████████| 469/469 [01:38<00:00,  4.74it/s]



Test set: epoch: 13, Average loss: 0.0302, Accuracy: 9937/10000 (99.4%)

